<a href="https://colab.research.google.com/github/ednadev/Data-Scientist/blob/master/05_Deep_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MNIST 손글씨 예측하기

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

#### 하이퍼파라미터, DataLoader

In [ ]:
input_size=784 # 28 x 28 x 1
hidden_size=500 # 784를 다시 500개로 줄이고
num_classes = 10 # 최종적으로 10개가 출력
num_epochs=5
batch_size=100
learning_rate = 0.001

In [ ]:
train_dataset=torchvision.datasets.MNIST(root='../../data', 
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

'''
2단계에 걸쳐서 데이타셋을 로딩해온다
DataLoader 내부적으로 멀티쓰레딩 방식으로 랩핑되어 있어서 클래스를 빠르게 로더
한번할때마다 앞에서 정해준것처럼 100개씩 로딩...
shuffle = True는 로딩할때마다 데이타의 순서를 바꿔서 로딩
모델이 답지를 외우는 것을 방지하는 효과
'''
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False) # Test 할때는 필요없다

### 모델 생성하기 -- Fully Connected Network

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)

    return out #여기까지가 클래스 정의부분

### 인스턴스화

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader) # 600번 학습시킬것이다

for epoch in range(num_epochs): # 5
  for i, (images, labels) in enumerate(train_loader): #100개씩 train_loader에 있는
  # 이미지랑 라벨이랑 묶어서 반환(for문 한번 돌때마다 100장씩 로딩)
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    # Forward
    preds=model(images)
    loss = loss_function(preds, labels) # 두 라벨을 비교...loss를 구한다.

    # Backward
    optimizer.zero_grad() #backward 하기전에 무조건 초기화
    loss.backward() # 각각의 weight들이 loss에 대해서 얼마나 책임을 져야하는지..정량화
    optimizer.step() # 실질적인 weight들의 업데이트가 진행된다...학습

    if(i+1) % 100 == 0:
      print('Epoch [ {}/{} ], Step[{}/{}], Loss : {:.3f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

## Model Test

In [ ]:
with torch.no_grad(): # 미분하지 않겠다...학습하지 않겠다..backward x
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1) # 80(x), 3 ..80은 필요하지 않으므로 버리기 

    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Accuracy of the FCN images : {} %'.format(100 * correct/total))

In [ ]:
torch.save(model.state_dict(), 'model.ckpt') # 모델의 weight

In [ ]:
images.shape

In [ ]:
plt.imshow(images[1].cpu().reshape(28, 28))
print(outputs[1].argmax())

In [ ]:
outputs.shape

In [ ]:
for i in range(10):
  plt.figure(figsize=(2,2))
  plt.imshow(images[i].cpu().numpy().reshape(28,28)) 
  print(outputs[i].argmax())